In [29]:
import random

In [51]:
# Skipping to implement Problem 3.1 here. It's a design problem. The book mentioned that it's too long for an interview.

# Solution for problem 3.2

def create_stack_from_list(values, capacity=None):
    if capacity is None:
        capacity = len(values) * 2
    stack = Stack(capacity)
    list(map(stack.push, values))
    return stack

def create_stack_random(start=1, end=15, size=10, capacity=None):
    values = [random.randint(start, end) for _ in range(size)]
    return create_stack_from_list(values, capacity)

class Stack(list):

    class StackObject:
        def __init__(self, value):
            self.value = value
            self.next = None
            self.min = None
            
        def __repr__(self):
            return "{}".format(self.value)

    def __init__(self, capacity):
        self.capacity = capacity
        self.top = None
        self.size = 0
    
    def push(self, value):
        if self.capacity <= self.size:
            raise ValueError("Stack is full")
        
        obj = Stack.StackObject(value)
        if self.size == 0:
            obj.min = obj.value
            self.top = obj
        else:
            obj.next = self.top
            obj.min = min(self.top.min, obj.value) 
            self.top = obj

        self.append(obj)
        self.size += 1

    def min(self):
        return self.peek().min

    def peek(self):
        return self.top

    def pop(self):
        if self.size == 0:
            raise ValueError("Stack is empty")

        t = self.top
        self.top = self.top.next
        self.size -= 1
        return t

# test part

values = [9, 7, 3, 1, 7, 2, 0, 9]
stack = create_stack_from_list(values)
assert stack.min() == 0
stack.pop()
assert stack.min() == 0
stack.pop()
assert stack.min() == 1
assert stack.size == 6

In [65]:
# Solution for problem 3.3

class MultiStack:
    def __init__(self, capacity):
        self.capacity = capacity
        self.stack = [[]] # one element.
        self.num_stack = 1
    
    def peek(self):
        return self.stack.peek().peek()
    
    def push(self, e):
        s = self.stack[-1]
        if len(s) == self.capacity:
            # new stack creation
            self.stack.append([e])
            self.num_stack += 1
        else:
            s.append(e)

    def pop(self):
        s = self.stack[-1]
        if len(self.stack) == 1:
            if len(s) == 0:
                raise ValueError("Underflow")
        
        e = s.pop()
        if len(s) == 0 and len(self.stack) >= 2:
            self.stack.pop()  # remove the empty stack unless it's the only stack.
            self.num_stack -= 1

        return e

    def popAt(self, i):
        n = self.num_stack
        if n <= i:
            raise ValueError("Overflow")
        
        num_element_pushed = (n-i-2) * self.capacity + len(self.stack[-1])
        temp_stack = MultiStack(self.capacity)
        for j in range(num_element_pushed):
            temp_stack.push(self.pop())
        
        e = self.pop()
        
        for j in range(num_element_pushed):
            self.push(temp_stack.pop())

        return e
            
    def __repr__(self):
        return f"{self.stack}"
    
    def __len__(self):
        return len(self.stack)

stack = MultiStack(capacity=3)
list(map(stack.push, range(13)))
assert len(stack) == 5
stack.pop()
assert len(stack) == 4
stack = MultiStack(capacity=3)
list(map(stack.push, range(13)))
print(stack)
stack.popAt(0)
print(stack)
stack.popAt(0)
print(stack)
stack.popAt(1)
print(stack)

[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11], [12]]
[[0, 1, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
[[0, 1, 4], [5, 6, 7], [8, 9, 10], [11, 12]]
[[0, 1, 4], [5, 6, 8], [9, 10, 11], [12]]


In [93]:
# Solution for problem 3.4

class Queue:
    def __init__(self):
        self.temp_stack = []
        self.item_stack = []
        self.__num_items = 0

    def add(self, e):
        Queue.transfer_items(self.item_stack, self.temp_stack)
        self.temp_stack.append(e)
        Queue.transfer_items(self.temp_stack, self.item_stack)
        self.__num_items += 1

    def peek(self):
        return self.item_stack[-1] if self.size() > 0 else None

    def remove(self):
        e = None
        if self.size() > 0:
            e = self.item_stack.pop()
            self.__num_items -= 1
        return e

    def size(self):
        return self.__num_items
    
    def __repr__(self):
        return f'{self.item_stack}'

    @staticmethod
    def transfer_items(full, empty):
        assert len(empty) == 0
        n = len(full)
        for _ in range(n):
            empty.append(full.pop())
            
q = Queue()
q.add(1)
q.add(2)
assert q.remove() == 1
q.add(3) 
assert q.peek() == 2
assert q.size() == 2
q = Queue()
assert q.peek() == None
assert q.remove() == None
q.add(6)

Second solution for 3.4

In [113]:
# Solution for problem 3.4
class Queue:
    def __init__(self):
        self.stack_oldest = []
        self.stack_newest = []
        self.size = lambda: len(self.stack_oldest) + len(self.stack_newest)

    def add(self, e):
        self.stack_newest.append(e)

    def peek(self):
        self.shift()
        return self.stack_oldest[-1]

    def remove(self):
        self.shift()
        return self.stack_oldest.pop()

    def shift(self):
        if len(self.stack_oldest) == 0:
            while len(self.stack_newest) != 0:
                self.stack_oldest.append(self.stack_newest.pop())
                
    def __repr__(self):
        return f'stack_old: {self.stack_oldest}\tstack_new: {self.stack_newest}'

q = Queue()
q.add(1)
q.add(5)
print(q)
q.peek()
print(q)
q.add(10)
q.add(20)
print(q)
q.remove()
print(q)
q.remove()
print(q)
assert q.peek() == 10
print(q)

stack_old: []	stack_new: [1, 5]
stack_old: [5, 1]	stack_new: []
stack_old: [5, 1]	stack_new: [10, 20]
stack_old: [5]	stack_new: [10, 20]
stack_old: []	stack_new: [10, 20]
stack_old: [20, 10]	stack_new: []


In [133]:
# Solution for problem 3.5

import sys
from random import randint

MAXIMUM = -sys.maxsize

class Stack:
    def __init__(self):
        self.stack = []

    def __repr__(self):
        return f'{self.stack}'

    def push(self, e):
        self.stack.append(e)
        
    def pop(self):
        return self.stack.pop()
    
    def sort(self):
        temp = []
        n = len(self.stack)

        for i in range(n):
            # search for max.
            maximum = MAXIMUM
            for _ in range(n-i):
                e = self.stack.pop()
                if maximum < e:
                    maximum = e
                temp.append(e)
            
            # leave max in temp.
            is_added = False
            for _ in range(n-i):
                e = temp.pop()
                if e == maximum and not is_added:
                    is_added = True
                else:
                    self.stack.append(e)

            temp.append(maximum)

        assert len(self.stack) == 0 and len(temp) == n
        self.stack = temp
        
s = Stack()
list(map(s.push, [randint(1, 20) for _ in range(10)]))
print(s)
s.sort()
print(s)
assert sorted(s.stack, reverse=True) == s.stack 

[19, 7, 10, 19, 4, 9, 19, 11, 8, 7]
[19, 19, 19, 11, 10, 9, 8, 7, 7, 4]


In [31]:
# Solution for problem 3.6

class Node:
    def __init__(self, value, t):
        self.t = t
        self.value = value
        self.offset = None
        
    def __repr__(self):
        return f'{self.value}|{self.t}|{self.offset}'

class Queue:

    def __init__(self):
        self.dog = []
        self.cat = []

    def __repr__(self):
        return f'{self.cat}->{self.dog}'

    @staticmethod
    def get_next_offset_queue(q):
        return 1 if len(q) == 0 else q[-1].offset + 1
    
    def next_offset(self):
        return max(Queue.get_next_offset_queue(self.dog), Queue.get_next_offset_queue(self.cat))
    
    def enqueue(self, e):
        e.offset = self.next_offset()
        if e.t == 'dog':
            self.dog.append(e)
        else:
            self.cat.append(e)

    def get_older_queue(self):
        if len(self.dog) == 0 and len(self.cat) == 0:
            raise ValueError("Queues are empty")
        elif len(self.dog) != 0 and len(self.cat) != 0:
            if self.dog[0].offset < self.cat[0].offset:
                return self.dog
            else:
                return self.cat
        else:
            if len(self.dog) == 0:
                return self.cat
            else:
                return self.dog
                
    def peek(self):
        return self.get_older_queue()[0]
    
    def dequeue(self):
        return self.get_older_queue().pop(0)
       
    def dequeue_dog(self):
        return self.dog.pop()

    def dequeue_cat(self):
        return self.dog.pop()
    

q = Queue()  
q.enqueue(Node(1, 'dog'))
q.enqueue(Node(2, 'cat'))
q.enqueue(Node(3, 'cat'))
q.enqueue(Node(4, 'dog'))
print(q)
print(q.peek())
q.dequeue()
q

[2|cat|2, 3|cat|3]->[1|dog|1, 4|dog|4]
1|dog|1


[2|cat|2, 3|cat|3]->[4|dog|4]

In [124]:
# Solution (not done) for problem 3.6


class Queue:

    def __init__(self):
        self.newer = []
        self.older = []
        self.first_dog = None
        self.last_dog = None
        self.first_cat = None
        self.last_cat = None

    def __repr__(self):
        return f'{self.older}->{self.newer}'

    def enqueue(self, e):
        if e.t = 'dog':
            self.enqueue_dog(e)
        else:
            self.enqueue_cat(e)

    def enqueue_dog(self, e):
        if self.first_dog is None:
            self.first_dog = self.last_dog = e
            self.newer.append(e)
        else:
            self.last_dog.next = e
            self.e.prev = self.last_dog
            self.last_dog = e
            self.e.next = None

    def enqueue_cat(self, e):
        if self.first_cat is None:
            self.first_cat = self.last_cat = e
            self.newer.append(e)
        else:
            self.last_cat.next = e
            self.e.prev = self.last_cat
            self.last_cat = e
            self.e.next = None

    def peek(self):
        if len(self.older) > 0:
            e = self.older[0]
        else:
            e = self.newer[0]
        return e

    def remove(self):
        if len(self.older) > 0:
            e = self.remove(0)
        else:
            e = self.remove(0)
        return e

    def dequeue(self):
        e = self.peek()
        if e.type == 'dog':
            self.dequeue_dog()
        else:
            self.dequeue_cat()
    

    def dequeue_dog(self):
        e = self.peek()
        if e.type == 'dog':
            self.remove()
        else:
            # top has some cats
            if len(self.older) == 0:
                e = self.remove()
                self.older.append(e)
                while True: 
                    e = self.newer.pop()
                    if len(self.newer) !=0 or e.type != 'dog':
                        break
                    else:
                        self.older.append(e)
            else:
                ...

        # adjust linked-list pointers.
        # then return.
        return e

    def dequeue_cat(self):




-9223372036854775807